In [ ]:
import numpy as np 
import math 
import rospy 
from tf import transformations
from std_msgs.msg import Header, ColorRGBA
from visualization_msgs.msg import Marker, MarkerArray
import sys
sys.path.append("..")
from kinematics.collision_checker import PyFCL
# from kinematics.robot import RobotClass 
from kinematics.rviz import RvizClass
print("Done.")

### Instance 

In [ ]:
file_name = "../urdf/ur5e/ur5e_onrobot.urdf"
base_offset = [0.18,0.,0.79]
pyfcl = PyFCL(verbose=False)
rviz  = RvizClass(file_name=file_name, base_offset=base_offset)



In [ ]:
import numpy as np 
import sys
sys.path.append("..")
from kinematics.utils.util_ik import make_ik_input
from kinematics.utils.util_structure import *
from kinematics.structure import CHAIN
from kinematics.collision_checker import PyFCL

class RobotClass(object):
    def __init__(self, file_name, base_offset, ctrl_joint_num=7):
        super(RobotClass,self).__init__() 
        self.chain = CHAIN(file_name=file_name, base_offset=base_offset)
        self.chain.add_joi_to_robot()
        self.chain.add_link_to_robot()
        self.chain.fk_chain(1)
        self.ctrl_joint_num = ctrl_joint_num
        self.fclize = PyFCL(verbose=False) 

    def solve_ik(self,  target_name      = ['wrist_3_joint'],
                        target_position  = [[0,0,0]],
                        target_rotation  = [[-1.57, 0, -1.57]],
                        solve_position   = [1],
                        solve_rotation   = [1],
                        weight_position  = 1,
                        weight_rotation  = 1,
                        joi_ctrl_num= 6): 
                        
        ingredients = self.make_ik_input(target_name = target_name,
                                    target_pos  = target_position,
                                    target_rot  = target_rotation,
                                    solve_pos   = solve_position,
                                    solve_rot   = solve_rotation,
                                    weight_pos  = weight_position,
                                    weight_rot  = weight_rotation,
                                    joi_ctrl_num= joi_ctrl_num)
        total_q = self.chain.get_q_from_ik(ingredients)
        ctrl_q  = total_q[:joi_ctrl_num+1] #Including base joint for Unity Env
        ctrl_q  = ctrl_q.reshape(-1,)
        return ctrl_q 

    def make_ik_input(self, target_name=["base_joint"], 
                    target_pos=[[0, 0, 0]], 
                    target_rot=[[0, 0, 0]], 
                    solve_pos=[1], 
                    solve_rot=[1], 
                    weight_pos=1, 
                    weight_rot=0, 
                    disabled_joi_id=[], 
                    joi_ctrl_num=7):
        return {"target_joint_name":target_name, 
                "target_joint_position":target_pos,
                "target_joint_rotation":target_rot,
                "solve_position":solve_pos,
                "solve_rotation":solve_rot,
                "position_weight":weight_pos,
                "rotation_weight":weight_rot,
                "disabled_joi_id":disabled_joi_id, 
                "joint_ctrl_num":joi_ctrl_num}

    def self_collision_check(self, q_list): 
        update_q_chain(self.chain.joint, q_list, self.ctrl_joint_num)
        self.chain.fk_chain(1)
        cap_R_list = get_cap_R_chain(self.chain.link)
        rpy_list = get_rpy_from_R_mat(cap_R_list)
        link_p_list = get_center_p_chain(self.chain.link)
        name_list = get_name_chain(self.chain.link)
        size_list = get_cap_size_chain(self.chain.link)
        scale_list = get_scale_chain(self.chain.link)                  
        height_list = get_height_chain(self.chain.link)
        radius_list = get_cap_radius(self.chain.link)
        # Capsulation 
        fcl_ingredients = get_cc_ingredients(name_list, link_p_list, rpy_list, height_list, radius_list)
        fclized_obj_lst = []
        for ingred in fcl_ingredients: 
            if ingred["name"] == "mount_link" or ingred["name"] == "camera_base_link" or ingred["name"] == "gripper_base_link": 
                pass 
            else: 
                fclized_obj = self.fclize.fclize_obj(ingred)
                fclized_obj_lst.append(fclized_obj)
        camera_link = {"name":"camera_base_link", 
                       "type":"box",
                       "position":robot.chain.link[8].cap.p.reshape(3,).tolist(),
                       "orientation":decompose_rotation_matrix(self.chain.joint[8].R).tolist(),
                       "size":[0.09, 0.003, 0.003]}
        fclized_camera = self.fclize.fclize_obj(camera_link)
        for idx, obj in enumerate(fclized_obj_lst): 
            col = self.fclize.one2one_cc(obj, fclized_camera)
            if col:
                print("Is collision?", col, fcl_ingredients[idx]["name"])
        for idx1, obj1 in enumerate(fclized_obj_lst): 
            for idx2, obj2 in enumerate(fclized_obj_lst):
                if idx1==idx2 or abs(idx1-idx2)<2 or idx1>8 or idx2>8: 
                    pass 
                else: 
                    col = self.fclize.one2one_cc(obj1, obj2)
                    if col:
                        print("Is collision?", col, fcl_ingredients[idx1]["name"]+" collision with "+fcl_ingredients[idx2]["name"])
    # def is_collision(self, q_list, objs=None):
    #     update_q_chain(self.chain.joint, q_list, self.ctrl_joint_num)
    #     self.chain.fk_chain(1)
    #     cap_R_list = get_cap_R_chain(self.chain.link)
    #     rpy_list = get_rpy_from_R_mat(cap_R_list)
    #     link_p_list = get_center_p_chain(self.chain.link)
    #     name_list = get_name_chain(self.chain.link)
    #     size_list = get_cap_size_chain(self.chain.link)
    #     scale_list = get_scale_chain(self.chain.link)                  
    #     height_list = get_height_chain(self.chain.link)
    #     radius_list = get_cap_radius(self.chain.link)
    #     for name, position, height, radius in zip(name_list, link_p_list, height_list, radius_list):
    #         print("name:{}, position:{}, height:{}, radius:{}".format(name, position, height, radius))
    #     # Capsulation 
    #     fcl_ingredients = get_cc_ingredients(name_list, link_p_list, rpy_list, height_list, radius_list)
    #     fclized_obj_lst = []
    #     for ingred in fcl_ingredients: 
    #         fclized_obj = self.fclize.fclize_obj(ingred)
    #         fclized_obj_lst.append(fclized_obj)
    #     print("total",len(fclized_obj_lst))

    #     # for idx, ingred in enumerate(fcl_ingredients):
    #     #     if idx == len(fcl_ingredients)-1:
    #     #         break 
    #     #     fclized_obj = fclized_obj_lst[idx]
    #     #     col = self.fclize.one2many_cc(fclized_obj, fclized_obj_lst[idx+2:])
    #     #     print("Is collision?", col)

    #     for idx1, ingred in enumerate(fcl_ingredients):
    #         if idx1 >8:
    #             break 
    #         print(fcl_ingredients[idx1]["name"])
    #         if fcl_ingredients[idx1]["name"] =="mount_link":
    #             pass 
    #         else: 
    #             fclized_obj1 = fclized_obj_lst[idx1]
    #             for idx2, fclized_obj2 in enumerate(fclized_obj_lst):
    #                 if idx2 >8:
    #                     break 
    #                 if fcl_ingredients[idx2]["name"] =="mount_link":
    #                     pass 
    #                 else: 
    #                     if idx1==idx2 or idx2==idx1+1 or idx2 ==idx1-1: 
    #                         col = False  
    #                     else:
    #                         col = self.fclize.one2one_cc(fclized_obj1, fclized_obj2)
    #                     if col:
    #                         print("Is collision?", col, fcl_ingredients[idx1]["name"]+" collision with "+fcl_ingredients[idx2]["name"])

In [ ]:
robot = RobotClass(file_name=file_name, base_offset=base_offset)

q = robot.solve_ik(target_name = ['mount_camera_joint'],
            target_position  = [[0.0,0.,1.2]],
            target_rotation  = [[-np.deg2rad(-50), +np.deg2rad(-90), +np.deg2rad(-90)]],
            solve_position   = [1],
            solve_rotation   = [1],
            weight_position  = 1,
            weight_rotation  = 1,
            joi_ctrl_num= 6)

q_list =q#np.array([0,0,0,0,0,0,0])
robot.self_collision_check(q_list)


tcp_position    = robot.chain.joint[10].p.reshape(3,).tolist()
tcp_rotation =  decompose_rotation_matrix(robot.chain.joint[10].R).tolist()
camera_position = robot.chain.link[8].cap.p.reshape(3,).tolist()
camera_rotation = decompose_rotation_matrix(robot.chain.joint[8].R).tolist()

obs_info  = rviz.make_markers(name="tcp_marker", 
                         type="cube",  
                         pos=tcp_position, 
                         rot=tcp_rotation, 
                         size=[0.025, 0.025, 0.025], 
                         color=[1,0,0,0.5])

obs_info2  = rviz.make_markers(name="camera_marker", 
                         type="cube",  
                         pos=camera_position, 
                         rot=camera_rotation, 
                         size=[0.090, 0.003, 0.003], 
                         color=[0,1,0,1])

obs_info3  = rviz.make_markers(name="RobotTable", 
                         type="cube",  
                         pos=[0,0,0.79/2], 
                         rot=[0,0,0], 
                         size=[0.76,0.72,0.79], 
                         color=[0,0,0,0.2])

obs_info4  = rviz.make_markers(name="Table", 
                         type="cube",  
                         pos=[0.76/2+1.2/2,0,0.74/2], 
                         rot=[0,0,0], 
                         size=[1.2,0.9,0.74], 
                         color=[0,0,0,0.2])
obs_info_lst = [obs_info, obs_info2, obs_info3, obs_info4]      

START = True
""" VISUALIZE """
if START:
    rendering=0
    rate = rospy.Rate(50)
    while not rospy.is_shutdown():
        if rendering == 1000:
            break 
        rviz.publish_robot(q_list=q_list)
        rviz.publish_markers(obs_info_lst)
        rendering +=1
        rate.sleep()